In [ ]:
import numpy as np

from experiments.utils.SimpleComparison import SimpleComparison
from hdimvis.algorithms.spring_force_algos.chalmers96_algo.Chalmers96 import Chalmers96
from hdimvis.visualise_layouts_and_metrics.plot import show_layout,show_generation_metrics
from hdimvis.algorithms.stochastic_ntet_algo.SNeD import SNeD
from hdimvis.create_low_d_layout.LayoutCreation import LayoutCreation
from hdimvis.data_fetchers.DataFetcher import DataFetcher
from hdimvis.visualise_layouts_and_metrics.plot import show_layout
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from experiments.utils.get_kmeans_f1 import get_kmeans_f1
from experiments.utils.get_f1_for_best_k_with_knn import get_f1_for_best_k_with_knn
from experiments.utils.get_avg_classwise_f1 import get_avg_classwise_f1
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from pathlib import Path
from definitions import PROJECT_ROOT
import pickle
from scipy import stats
import matplotlib.pyplot as plt
from time import perf_counter
from  sklearn.manifold import SpectralEmbedding, MDS
from hdimvis.metrics.distance_measures.euclidian_and_manhattan import euclidean
import pandas as pd

In [ ]:
dataset= DataFetcher.fetch_data("metro")

In [4]:
results = [[],[]]
num_repeats = 3

In [5]:
Xld = PCA(n_components=2, whiten=False, copy=True).fit_transform(dataset.data).astype(np.float64)
Xld *= 10/np.std(Xld)

for i in range(num_repeats):

    no_knnd_start = perf_counter()
    algo96 = Chalmers96(dataset=dataset, distance_fn=euclidean,
             spring_constant=0.05, initial_layout=Xld,
            use_knnd=False)
    layout = LayoutCreation.create_layout(algo96, no_iters=100)
    results[0].append(perf_counter() - no_knnd_start)



    knnd_start = perf_counter()
    algo96 = Chalmers96(dataset=dataset, distance_fn=euclidean,
             spring_constant=0.05, initial_layout=Xld,
            use_knnd=True)
    layout_knnd = LayoutCreation.create_layout(algo96, no_iters=100)
    results[1].append(perf_counter() - knnd_start)



####################
A 2D layout of the "metro" dataset will be created 
using the "Chalmers' 1996" algorithm
####################
The HD distance measure used is: euclidean
####################
####################
No metrics will be collected during layout creation. 
To change this use the 'metric collection' parameter of the layout 
####################
All stress calculations will be performed using the euclidian norm
####################
Spring constant is set to  0.05 
Damping constant is set to 0 
Spring constant scaling factor is set to 0.03333333333333333 
####################
The algorithm will use a k-nearest neighbours graph generated by the k-NN Descent algorithm
####################
A 2D layout of the "metro" dataset will be created 
using the "Chalmers' 1996" algorithm
####################
The HD distance measure used is: euclidean
####################
####################
No metrics will be collected during layout creation. 
To change this use the 'metric collection' pa

KeyboardInterrupt: 

In [6]:
print(results)

[[1602.6357605999801, 1617.7469972999534], [21481.32037490001]]
